In [1]:
import warnings
warnings.filterwarnings("always")
warnings.filterwarnings("ignore")

In [2]:
import numpy as np
import pandas as pd 
import pydicom
import cv2
import os
import collections
from tqdm import tqdm_notebook as tqdm
from datetime import datetime
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, Activation
from keras.optimizers import RMSprop, Adam
from keras.utils import to_categorical, Sequence
from keras.applications import VGG19, VGG16, ResNet50

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [3]:
data_path = "data/"

In [5]:
train_df = pd.read_csv(data_path + "stage_2_train.csv")
train_df.head()

,ID,Label
0,ID_12cadc6af_epidural,0
1,ID_12cadc6af_intraparenchymal,0
2,ID_12cadc6af_intraventricular,0
3,ID_12cadc6af_subarachnoid,0
4,ID_12cadc6af_subdural,0


In [6]:
train_df["sub_type"] = train_df["ID"].apply(lambda x:x.split("_")[-1])
train_df["file_name"] = train_df["ID"].apply(lambda x: "_".join(x.split("_")[:2])+ ".dcm")

In [7]:
train_df.head()

,ID,Label,sub_type,file_name
0,ID_12cadc6af_epidural,0,epidural,ID_12cadc6af.dcm
1,ID_12cadc6af_intraparenchymal,0,intraparenchymal,ID_12cadc6af.dcm
2,ID_12cadc6af_intraventricular,0,intraventricular,ID_12cadc6af.dcm
3,ID_12cadc6af_subarachnoid,0,subarachnoid,ID_12cadc6af.dcm
4,ID_12cadc6af_subdural,0,subdural,ID_12cadc6af.dcm


In [8]:
train_df.drop(train_df[train_df["file_name"] == "ID_6431af929.dcm"].index, inplace= True)

In [9]:
train_df= pd.pivot_table(train_df.drop(columns = "ID"), index= "file_name", columns = "sub_type", values = "Label")

In [10]:
train_df.head()

sub_type,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural
file_name,,,,,,
ID_000012eaf.dcm,0,0,0,0,0,0
ID_000039fa0.dcm,0,0,0,0,0,0
ID_00005679d.dcm,0,0,0,0,0,0
ID_00008ce3c.dcm,0,0,0,0,0,0
ID_0000950d7.dcm,0,0,0,0,0,0
